In [19]:
import pandas as pd
import gzip

#f = gzip.open("/data/tv/2016/2016-01/2016-01-18/2016-01-18_0100_US_MSNBC_US_Presidential_Politics.v4.vrt.gz", "r")
f = gzip.open("tmp/2016-01-18_1500_US_CNN_Newsroom.v4.vrt.gz", "rt")
# read file and delete lines starting with "<"
lines = f.readlines()
lines = [line for line in lines if not line.startswith('<')]
f.close()
# create a list from lines using "\t" as separator
lines = [line.split("\t") for line in lines]
lines

gentle = pd.DataFrame(lines)


filtrado = df.iloc[:, [0, 51, 52 ]]

filtrado.head(20)

,0,51,52
0,Half,0,0
1,",",0,0
2,scored,0,0
3,24,0,0
4,straight,0,0
5,points,0,0
6,",",0,0
7,put,0,0
8,them,0,0
9,within,0,36


In [ ]:
speech2 = pd.read_csv("2016-01-18_1500_US_CNN_Newsroom.v4.vrt.gz", sep = "\t", header = 6, skiprows = 6, decimal = ".", encoding = "UTF-8")
speech2.head(20)

In [ ]:
# delete rows with first column start with "<" 
#df[df[1].str.contains("this string")==False]
df.columns





#filtrado = speech.iloc[:, [0, 51, 52 ]]
# create new column with column 1 + column 2 * 0.001
# rename columns to "string", "second" and "milisecond"
#filtrado.columns = ["string", "second", "milisecond"]
#filtrado["cutTime"] = filtrado["second"] + filtrado["milisecond"] * 0.001
#filtrado.head(20)


In [ ]:
# print rows with string isin ["whole", "debate"]
print(filtrado[filtrado["string"].isin(["whole"])])

In [ ]:
# get rows with conditions
cond = filtrado["string"].eq("whole") & filtrado["string"].shift(-1).eq("debate")

In [ ]:
cortes = filtrado[cond]
cortes

In [ ]:
# Create new column in cortes called "cutTime" with the value of second + milisecond * 0.001. 
#cortes["cutTime"] = cortes["second"] + (cortes["milisecond"] * 0.001)
cortes["milisecond"] = cortes["milisecond"] * 0.001
#cortes["cutTime"] = cortes.apply(lambda x: x.second + (x.milisecond * 0.001), axis = 1)
cortes["cutTime"] = cortes["second"] + cortes["milisecond"]
cortes


In [ ]:
a = list(cortes["cutTime"])

In [ ]:
print(a)


In [ ]:

# get cut times of a video containing 2 word searching text
def get_cut_times(video, search_text):
  gentle = pd.read_csv(video, sep = "\t", header = 6, skiprows = 6, decimal = ".", encoding = "UTF-8")
  # Selection on important columns
  selection1 = gentle.iloc[:, [0, 51, 52 ]]
  # rename columns to "string", "second" and "milisecond"
  selection1.columns = ["string", "second", "milisecond"]
  word1 = search_text.split(" ")[0]
  word2 = search_text.split(" ")[1]
  # Selection on rows with string isin [word1, word2]
  cond = selection1["string"].eq(word1) & selection1["string"].shift(-1).eq(word2)
  cutting_times = filtrado[cond]
  # Create new column in cutting_times called "cutTime" with the value of second + milisecond * 0.001.
  cutting_times["cutTime"] = cutting_times["second"] + (cutting_times["milisecond"] * 0.001)
  return list(cutting_times["cutTime"])


get_cut_times("2016-01-18_1500_US_CNN_Newsroom.v4.vrt.gz", "whole debate")


In [53]:

import pandas as pd

# get cut times of a video containing n word searching text and return list with exact second 
# where the first word of text is found.
# Imagine this gentle output:
# [1st match "whole debate"]
#  string  Time
# whole   2125.059
# debate  2125.088
# was     2126.030
#
## [2nd match "whole debate"]
# string  Time
# whole   2133.004
# debate  2133.032
# ,       2133.077
# 
# [3]
# string  Time
# whole   2150.060
# debate  2150.089
# rises   2151.030

# example: get_cut_times("2016-01-18_1500_US_CNN_Newsroom.v4.vrt.gz", "whole") - return: [398.039, 1800.052, 2125.059, 2133.004, 2150.06, 2483.044, 2675.048]
# example: get_cut_times("2016-01-18_1500_US_CNN_Newsroom.v4.vrt.gz", "whole debate") - return: [2125.059, 2133.004, 2150.06]
# example: get_cut_times("2016-01-18_1500_US_CNN_Newsroom.v4.vrt.gz", "whole debate rises") - return: [2150.06]
def get_cut_times(video, search_text):
  cutting_times = {}
  try:
    # read file and delete lines starting with "<"
    f = gzip.open(video,"rt")
    lines = f.readlines()
    lines = [line for line in lines if not line.startswith('<')]
    f.close()
    # create a list from lines using "\t" as separator
    lines = [line.split("\t") for line in lines]
    gentle = pd.DataFrame(lines)
    # Selection on important columns
    selection1 = gentle.iloc[:, [0, 51, 52 ]]
    # rename columns to "string", "second" and "milisecond"
    selection1.columns = ["string", "second", "milisecond"]
    words = search_text.split(" ")
    # get rows with conditions. consecutive words must be in consecutive rows
    for i in range(len(words)):
      if i == 0:
        cond = selection1["string"].eq(words[i])
      else:
        cond = cond & selection1["string"].shift(-1*i).eq(words[i])
    cutting_times = selection1[cond]
    # Create new column in cutting_times called "cutTime" with the value of second + milisecond * 0.001.
    cutting_times["second"]     = cutting_times["second"].apply(lambda x: float(x))
    cutting_times["milisecond"] = cutting_times["milisecond"].apply(lambda x: float(x) * 0.0001)
    cutting_times["cutTime"] = cutting_times["second"] + cutting_times["milisecond"]
  except:
    print("Error: " + video)
    cutting_times["cutTime"] = []
  return list(cutting_times["cutTime"])


print(get_cut_times("tmp/2016-01-18_1500_US_CNN_Newsroom.v4.vrt.gz", "whole debate"))
  
print("new game") 
print(get_cut_times("tmp/2016-09-01_2200_US_KNBC_The_Ellen_DeGeneres_Show.v4.vrt.gz", "new game"))
print("\n")
print("new game called")
print(get_cut_times("tmp/2016-09-01_2200_US_KNBC_The_Ellen_DeGeneres_Show.v4.vrt.gz", "new game called"))


print(get_cut_times("/data/tv/2016/2016-01/2016-01-18/2016-01-18_0100_US_MSNBC_US_Presidential_Politics.v4.vrt.gz", "new game called"))




/tmp/ipykernel_148786/1724239485.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cutting_times["second"]     = cutting_times["second"].apply(lambda x: float(x))
/tmp/ipykernel_148786/1724239485.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cutting_times["milisecond"] = cutting_times["milisecond"].apply(lambda x: float(x) * 0.0001)
/tmp/ipykernel_148786/1724239485.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexe

[2125.0059, 2133.0004, 2150.006]
new game
[122.0034, 482.0042]


new game called
[122.0034]
will stick tonight
[271.0037]


In [ ]:
def get_all_vrts(videos_folder):
    vrts = []
    for root, dirs, files in os.walk(videos_folder):
        for file in files:
            if file.endswith(".vrt.gz"):
                vrts.append(os.path.join(root, file))
    return vrts


# get all vrts in a folder

vrts = get_all_vrts("/data/tv/2016/2016-09")
vrts